In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import requests
import string

from tkinter.ttk import *
from tkinter import *
import tkinter
from io import BytesIO
from PIL import ImageTk, Image


In [2]:
def get_title(index):
    return df[df.Index == index]["Title"].values[0]

def get_index(title):
    return df[df.Title == title]["Index"].values[0]

In [3]:
df = pd.read_csv("imdb_movies.csv")
df.head(2)

,Title,Year,Ratings,Genre,Description,Director,Cast,Image,Duration,Certificate,Votes
0,Inception,2010,8.8,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...",https://m.media-amazon.com/images/M/MV5BMjAxMz...,148 min,UA,"19,89,040"
1,The Dark Knight Rises,2012,8.4,"Action, Adventure",Eight years after the Joker's reign of anarchy...,Christopher Nolan,"Christian Bale, Tom Hardy, Anne Hathaway",https://m.media-amazon.com/images/M/MV5BMTk4OD...,164 min,UA,"14,74,141"


In [4]:
all_movies="Sr.No.\tTitle\t\t\t\t\tGenre\t\t\t\t\tRatings\n\n"
for i in range(df.shape[0]):
    all_movies+=  str(i+1)+".\t"+df.loc[i]['Title']+" ("+str(df.loc[i]['Year'])+")\t\t\t\t\t"+df.loc[i]['Genre']+"\t\t\t\t\t"+str(df.loc[i]['Ratings'])+"\n"
# all_movies


In [5]:
l=[]
for i in range(df.shape[0]):
    l.append(i)
index = pd.Series(l)
df.insert(0, "Index", index, True)
df.set_index('Index')

gen=[]
for i in range(df.shape[0]):
    try:
        gen.append(set(df['Genre'][i].split(',')))
    except Exception as e:
        gen.append(set([]))


df['Ratings'] = df['Ratings']*10
df.Ratings = df.Ratings.astype(int) 
df.head(2)


,Index,Title,Year,Ratings,Genre,Description,Director,Cast,Image,Duration,Certificate,Votes
0,0,Inception,2010,88,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...",https://m.media-amazon.com/images/M/MV5BMjAxMz...,148 min,UA,"19,89,040"
1,1,The Dark Knight Rises,2012,84,"Action, Adventure",Eight years after the Joker's reign of anarchy...,Christopher Nolan,"Christian Bale, Tom Hardy, Anne Hathaway",https://m.media-amazon.com/images/M/MV5BMTk4OD...,164 min,UA,"14,74,141"


In [6]:
def combine_Features(l):
    if l==[]:
        features = ['Cast','Genre','Director','Votes']
    else:
        features=l


    for feature in features:
        df[feature] = df[feature].fillna('')
    def combine_features(row):
        comb=""
        for i in features:
            comb = comb + " " + str(row[i])
        return comb
    df["combined_features"] = df.apply(combine_features,axis=1)
#     print(df["combined_features"].head())
# combine_Features(['Genre'])

In [7]:
def SimilarityMatrix(l):
    combine_Features(l)
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(df["combined_features"])
    cosine_sim = cosine_similarity(count_matrix) 
    return cosine_sim

In [8]:
def similarMovies(movie_user_likes,l):
    cosine_sim = SimilarityMatrix(l)
    
    movie_index = get_index(movie_user_likes)
    
    similar_movies =  list(enumerate(cosine_sim[movie_index]))
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)
    l=[]
    for i in range(1,51):
        l.append(get_title(sorted_similar_movies[i][0]))
    return l

In [9]:
def genre(g):
    count = 0
    gbased = "Sr.No.\tTitle\t\t\t\t\tGenre\t\t\t\t\tRatings\n\n"
    try:
        for i in range(df.shape[0]):
            if g.intersection(gen[i]) != set():
                count+=1
                gbased+=str(count)+".\t"+df.loc[i]['Title']+" ("+str(df.loc[i]['Year'])+")\t\t\t\t\t"+df.loc[i]['Genre']+"\t\t\t\t\t"+str(df.loc[i]['Ratings'])+"\n"
        return gbased,count
    except:
        return "Invalid Genre!",count

def rating(r):
    r=r*10
    rbased="Sr.No.\tTitle\t\t\t\t\tGenre\t\t\t\t\tRatings\n\n"
    count = 0
    for i in range(df.shape[0]):
        if r == df.loc[i]['Ratings']:
            count+=1
            rbased+=str(count)+".\t"+df.loc[i]['Title']+" ("+str(df.loc[i]['Year'])+")\t\t\t\t\t"+df.loc[i]['Genre']+"\t\t\t\t\t"+str(df.loc[i]['Ratings']/10)+"\n"
    return rbased,count

def rating_above(r):
    r=r*10
    rAbased="Sr.No.\tTitle\t\t\t\t\tGenre\t\t\t\t\tRatings\n\n"
    count = 0
    for i in range(df.shape[0]):
        if r <= df['Ratings'][i]:
            count+=1
            rAbased+=str(count)+".\t"+df.loc[i]['Title']+" ("+str(df.loc[i]['Year'])+")\t\t\t\t\t"+df.loc[i]['Genre']+"\t\t\t\t\t"+str(df.loc[i]['Ratings']/10)+"\n"
    return rAbased,count

def year(y):
    print('Fnc')
    yr="Sr.No.\tTitle\t\t\t\t\tGenre\t\t\t\t\tRatings\n\n"
    count = 0
    for i in range(df.shape[0]):
        if y == df.loc[i]['Year']:
            count+=1
            yr+=str(count)+".\t"+df.loc[i]['Title']+" ("+str(df.loc[i]['Year'])+")\t\t\t\t\t"+df.loc[i]['Genre']+"\t\t\t\t\t"+str(df.loc[i]['Ratings']/10)+"\n"
    return yr,count

In [10]:
# genre(set(['Action']))

In [26]:

class MyButton:
    def __init__(self):
        self.f=Frame(root, height=1000, width=1600)
        self.font=('Times', 16)
        self.small=('Times', 12)
        self.heading=('Times', 20,'bold')
        self.Ptitle=('Times', 30,'bold')
        self.f.propagate(0)
        self.f.pack()
        
        self.Ptitle=Label(self.f, text="IMDB Movie Recomendation System", font=self.Ptitle, fg='red')
        self.Ptitle.place(x=450, y= 10 )
        

        self.b1=Button(self.f, text='Search', command=lambda: self.buttonClick(1), width=15, height=2,bg='aqua')
        self.b1.place(x=1250, y=100, width = 200, height= 50)
        self.b2=Button(self.f, text='Similar', command=lambda: self.buttonClick(2), width=15, height=2,bg='aqua')
        self.b2.place(x=1250, y=700, width = 200, height= 50)    
        self.e1=Entry(self.f, fg='black', bg='white',width=15, font=self.font)
        self.e1.place(x=50, y=100, width = 1100, height= 50)
        
        
        self.l=Label(self.f, text="Select Features", width=25, height=2, font=self.heading, fg='red')
        self.l.place(x=1250, y= 200, width = 200)
        self.l2=Listbox(self.f, height=6, selectmode=MULTIPLE, activestyle='dotbox', font=self.font, fg='green', bg='light grey')
        self.l2.place(x=1250, y= 250, width = 200)
        self.lst=['Genre', 'Director', 'Ratings', 'Cast', 'Votes','Year']
        for i in self.lst:
            self.l2.insert(END, i)
        self.l2.bind('<<ListboxSelect>>')
        self.make_filter()
        
        
        
    def make_filter(self):
        self.fil=Label(self.f, text="Apply Filter", font=self.font, fg='green')
        self.fil.place(x=1250, y= 420, width = 200)
        
        self.str1=StringVar()
        self.strlist=['Genre', 'Rating', 'Rating Above','Year']
        self.sp2=Spinbox(self.f, values=self.strlist, textvariable=self.str1, fg='black', bg='grey90',width=15, font=self.font)
        self.sp2.place(x=1250, y= 450, width = 200)
        
        self.e2=Entry(self.f, fg='black', bg='white', font=self.font)
        self.e2.place(x=1250, y=480, width = 200)
        
        self.fb=Button(self.f, text='Filter', command=lambda: self.buttonClick(4), width=15, height=2)
        self.fb.place(x=1250, y=510, width = 200) 
        
    
    def clear_filter(self):
        try:
            self.fil.destroy()
            self.fil=None
        except:
            pass
        try:
            self.sp2.destroy()
            self.sp2=None
        except:
            pass
        try:
            self.e2.destroy()
            self.e2=None
        except:
            pass
        try:
            self.fb.destroy()
            self.fb=None
        except:
            pass
        try:
            self.count.destroy()
            self.count=None
        except:
            pass
        
        
    def clear_scroll_view(self):
        try:
            self.blank0.destroy()
            self.blank0=None
        except:
            pass
        try:
            self.t.destroy()
            self.t=None
        except:
            pass
        try:
            self.s.destroy()
            self.s=None
        except:
            pass
        try:
            self.b3.destroy()
            self.b3=None
        except:
            pass
        try:
            self.count.destroy()
            self.count=None
        except:
            pass

    def scroll_view(self,s,c):

        self.clear_scroll_view()
        self.e1.delete(0, 'end')
        self.blank0=Label(self.f, text="", width=25, height=2, font=self.font,bg='white')
        self.blank0.place(x=50, y= 200, width = 1100,height=450)

        self.t=Text(self.f,font=self.font,  wrap=WORD)
        self.t.insert(END, s)
        self.t.place(x=50, y= 200, width = 1080,height=450)
        self.s=Scrollbar(self.blank0, orient=VERTICAL, command=self.t.yview)
        self.t.configure(yscrollcommand=self.s.set)
        self.s.pack(side=RIGHT, fill=Y)

        self.b3=Button(self.f, text='Home', command=lambda: self.buttonClick(3), width=15, height=2,bg='aqua')
        self.b3.place(x=50, y=700, width = 200, height= 50) 
        
        self.count=Label(self.f, text= str(c)+" Entries Found", font=self.font, fg='green')
        self.count.place(x=1250, y= 550, width = 200)
        
    def buttonClick(self,id):
        if id==1:

            self.buttonClick(3)
            s1=self.e1.get()

            
            
            
            if s1=="":
                return
            
            if s1.lower()=='all' or s1.lower() == 'all movies':
                
                self.scroll_view(all_movies,df.shape[0])
                return
                
            l=list(df[(df == s1).any(1)].stack()[lambda x: x != s1].unique())
            if l==[]:
                print('No Such Movie in DB :(')
                return
            else:
                
                self.blank=Label(self.f, text="", width=25, height=2, font=self.font,bg='white')
                self.blank.place(x=50, y= 200, width = 1100,height=450)
                
                try:
                    self.response = requests.get(l[7])
                    self.path = Image.open(BytesIO(self.response.content))
                    self.img = ImageTk.PhotoImage(self.path)
                    self.panel = Label(self.f, image = self.img)
                    self.panel.place(x=50,y=300)
                    self.title=Label(self.f, text=s1+" ("+str(l[1])+")", font=self.heading, fg='black',bg='white')
                    self.title.place(x=50, y= 200)
                    
    
                except:
                    return
                
                self.tag=Label(self.f, text=str(l[9])+" | "+str(l[8])+" | "+str(l[3]), font=self.font, fg='dark grey',bg='white')
                self.tag.place(x=50, y= 240)
                
                self.tag2=Label(self.f, text="Rating: "+str(l[2])+" | Votes: "+str(l[-2]), font=self.font, fg='black',bg='white')
                self.tag2.place(x=250, y= 300)
                
                self.m=Message(self.f, text=l[4], font= self.font, fg='black',bg='white',anchor= 'nw',width = 850,)
                self.m.place(x=250, y= 350, width = 900,height=300)

                self.cast=Label(self.f, text="Director: "+str(l[5])+" | Cast: "+str(l[6]), font=self.font, fg='black',bg='white')
                self.cast.place(x=250, y= 600)
                
                self.b3=Button(self.f, text='Home', command=lambda: self.buttonClick(3), width=15, height=2,bg='aqua')
                self.b3.place(x=50, y=700, width = 200, height= 50)  
            self.clear_filter()

            
            
#             print(s1)
        elif id==2:
            self.buttonClick(3)
            
            s1=self.e1.get()
            if s1=="":
                return
            l=list(df[(df == s1).any(1)].stack()[lambda x: x != s1].unique())
            if l==[]:
                print('No Such Movie in DB :(')
                return
            else:
                ind=[self.lst[i] for i in self.l2.curselection()]
                
                selec=""
                for i in ind:
                    if i=='Genre':
                        selec+=''.join('Genre\t: '+l[3]+"\n")
                    if i=='Director':
                        selec+=''.join('Direc\t: '+l[5]+"\n")
                    if i=='Ratings':
                        selec+=''.join('Ratings\t: '+str(l[2]/10)+"\n")
                    if i=='Cast':
                        selec+=''.join('Cast\t: '+l[6]+"\n")
                    if i=='Votes':
                        selec+=''.join('Votes\t: '+str(l[-2])+"\n")
                    if i=='Year':
                        selec+=''.join('Year\t: '+str(l[1]))
                        
                
                if len(ind)>0:
                    try:
                        self.selection.destroy()
                    except:
                        pass
                    try:
                        self.rec.destroy()
                    except:
                        pass
                    
                    self.rec=Label(self.f, text="Recomendatiom based on", font=self.small, fg='red',bg='white')
                    self.rec.place(x=1250, y= 420, width = 200)
                    
                    self.selection=Message(self.f, text=selec, font= self.small, fg='black',bg='white',anchor= 'nw',width = 200,)
                    self.selection.place(x=1250, y= 450, width = 200)
                else:
                    try:
                        self.selection.destroy()
                    except:
                        pass
                    try:
                        self.rec.destroy()
                    except:
                        pass
                
                
                l = similarMovies(s1,ind)
                
                self.blank2=Label(self.f, text="", width=25, height=2, font=self.font, fg='red',bg='white')
                self.blank2.place(x=50, y= 200, width = 1100,height=450)
                
                l1=l[:17]
                s=""
                for i in range(len(l1)):
                    s+=str(i+1)+". "+l1[i]+"\n"
                self.m0=Message(self.f, text=s, font= self.font, fg='black',bg='white',anchor= 'nw',width = 350,)
                self.m0.place(x=50, y= 200, width = 350,height=450)
                
                l2=l[17:34]
                s=""
                for i in range(len(l2)):
                    s+=str(i+18)+". "+l2[i]+"\n"
                self.m2=Message(self.f, text=s, font= self.font, fg='black',bg='white',anchor= 'nw',width = 350,)
                self.m2.place(x=400, y= 200, width = 350,height=450)

                l3=l[34:]
                s=""
                for i in range(len(l3)):
                    s+=str(i+35)+". "+l3[i]+"\n"
                self.m3=Message(self.f, text=s, font= self.font, fg='black',bg='white',anchor= 'nw',width = 350,)
                self.m3.place(x=800, y= 200, width = 350,height=450)
                
                
                self.b2=Button(self.f, text='Similar', command=lambda: self.buttonClick(2), width=15, height=2,bg='aqua')
                self.b2.place(x=1250, y=700, width = 200, height= 50) 
                self.b3=Button(self.f, text='Home', command=lambda: self.buttonClick(3), width=15, height=2,bg='aqua')
                self.b3.place(x=50, y=700, width = 200, height= 50)  
                
            self.clear_filter()
                


                
        
        elif id==3:
            
            try:
                self.title.destroy()
                self.title=None
            except:
                pass
            try:
                self.tag.destroy()
                self.tag=None
            except:
                pass
            try:
                self.tag2.destroy()
                self.tag2=None
            except:
                pass
            try:
                self.m0.destroy()
                self.m0=None
            except:
                pass
            try:
                self.m.destroy()
                self.m=None
            except:
                pass
            try:
                self.m1.destroy()
                self.m1=None          

            except:
                pass
            try:
                self.m2.destroy()
                self.m2=None
            except:
                pass
            try:
                self.m3.destroy()
                self.m3=None
            except:
                pass
            try:
                self.b3.destroy()
                self.b3=None
            except:
                pass
            try:
                self.blank.destroy()
                self.blank=None
            except:
                pass
            try:
                self.blank2.destroy()
                self.blank2=None
            except:
                pass
            try:
                self.cast.destroy()
                self.cast=None
            except:
                pass
            try:
                self.panel.destroy()
                self.panel=None
            except:
                pass
            try:
                self.selection.destroy()
                self.selection=None
            except:
                pass
            self.clear_scroll_view()
            self.clear_filter()
            self.make_filter()
            
            
            
        elif id ==4: 
        
            s1 = self.str1.get()
            e1 =self.e2.get()
            
            if s1=="":
                return
            flag=0
            try:
                r = float(e1)
                print('Rateeeeee')
                if s1=="Rating":
                    r,c = rating(r)
                elif s1=="Rating Above":
                    r,c = rating_above(r)
                if s1=="Rating" or s1=="Rating Above":
                    self.scroll_view(r,c)
                elif s1=="Year":
                    yr= int(e1)
                    yr,c=year(yr)
                    self.scroll_view(yr,c)
            except ValueError:
                flag=1
            except TypeError:
                flag=1
            if flag==1:
                flag=0
                print("hi")
                if s1=="Genre":
                    g = set(e1.strip().split(','))
                    g,c = genre(g)
                    self.scroll_view(g,c)
                    
                
                


root=Tk()
my=MyButton()
root.mainloop()